In [1]:
import pandas as pd
import random
import string
import numpy as np
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json
from sklearn.externals import joblib
from collections import defaultdict

/Users/JMX/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# data processing for RNN model
file = pd.read_csv("./data.csv", encoding='ISO-8859-1')
data = file.values.tolist()
master_data = []
for l in data:
    master_data.append([l[0], l[5]])
random.Random(73).shuffle(master_data)
smaller_data = master_data[:50000]
tokenizer = TweetTokenizer()
r = tokenizer.tokenize(smaller_data[0][1])
print(r)
counter = 0
for l in smaller_data:
    r = ' '.join([x  for x in tokenizer.tokenize(l[1]) if x[0] != '@'])
    smaller_data[counter][1] = r
    counter +=1
texts = [x[1] for x in smaller_data]
Y = []
for l in smaller_data:
    if(l[0] == 0 ):
        Y.append([0, 1])
    else:
        Y.append([1, 0])
max_fatures = 2000
feature_tokenizer = Tokenizer(num_words=max_fatures, split=' ')
feature_tokenizer.fit_on_texts(texts)
X = feature_tokenizer.texts_to_sequences(texts)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196  

['thought', 'the', 'Summer', 'Ball', 'was', 'awesome', '!', 'Well', 'done', 'everyone']


In [3]:
feature_tokenizer_cnn = Tokenizer(num_words=max_fatures, split=' ', oov_token='<unw>')
feature_tokenizer_cnn.fit_on_texts(texts)
X_cnn = feature_tokenizer_cnn.texts_to_sequences(texts)
X_cnn = pad_sequences(X)

In [4]:
master_data[0]

[4, 'thought the Summer Ball was awesome ! Well done everyone']

In [5]:
# SVM model and baseline model data processing 
smaller_data_punc = []
smaller_data = master_data[:300000]
# with punctuation 
counter = 0
for l in master_data[:300000]:
    r = ''.join([c for c in l[1].lower() if c !='@'])
    if(l[0] == 0):
        smaller_data_punc.append([0, r])
    else:
        smaller_data_punc.append([1, r])
    counter +=1
    
train_text = [x[1] for x in smaller_data[:200000]]

wordCount = defaultdict(int)
for l in train_text:
    l = l.split()
    if(len(l) == 0):
        continue
    wordCount[l[0]] += 1
    for i in range(1, len(l)):
        wordCount[l[i]] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:2000]]


unigram_frequency = defaultdict(float)
total_count = sum([i[0] for i in counts[:2000]])
for i in counts[:2000]:
    unigram_frequency[i[1]] = i[0]/total_count 
    
tweet = TweetTokenizer()


wordCountP = defaultdict(int)
for d in smaller_data_punc:
    review = d[1]
    tokens = tweet.tokenize(review)
    for w in tokens:
        wordCountP[w] += 1

countsP = [(wordCountP[w], w) for w in wordCountP]
countsP.sort()
countsP.reverse()

wordsP = [x[1] for x in countsP[:2000]]

wordIdP = dict(zip(wordsP, range(len(wordsP))))
wordSetP = set(wordsP)


# Calculating the most 2000 frequenct words frequncy in probalility in positive sentiment text
unigram_frequency_pos = defaultdict(float)
train_text_pos = [x[1] for x in smaller_data[:200000] if x[0] == 4]
for l in train_text_pos:
    l = l.split()
    if(len(l) == 0):
        continue
    for w in l:
        if w in words:
            unigram_frequency_pos[w] += 1


total_count = 0
for w in unigram_frequency_pos:
    total_count += unigram_frequency_pos[w]
for w in unigram_frequency_pos:
    unigram_frequency_pos[w] = unigram_frequency_pos[w]/total_count
positive_words = []
for w in unigram_frequency:
    positive_words.append([unigram_frequency_pos[w] - unigram_frequency[w], w])
positive_words.sort()
positive_words.reverse()
pos_words = [x[1] for x in positive_words][:200]

# Calculating the most 2000 frequenct words frequncy in probalility in negtive sentiment text
unigram_frequency_neg = defaultdict(float)
train_text_neg = [x[1] for x in smaller_data[:200000] if x[0] == 0]
for l in train_text_neg:
    l = l.split()
    if(len(l) == 0):
        continue
    for w in l:
        if w in words:
            unigram_frequency_neg[w] += 1
total_count = 0
for w in unigram_frequency_neg:
    total_count += unigram_frequency_neg[w]
for w in unigram_frequency_neg:
    unigram_frequency_neg[w] = unigram_frequency_neg[w]/total_count
negative_words = []
for w in unigram_frequency:
    negative_words.append([unigram_frequency_neg[w] - unigram_frequency[w], w])
negative_words.sort()
negative_words.reverse()
neg_words = [x[1] for x in negative_words][:200]

In [6]:
pos_words

['you',
 '!',
 'a',
 'the',
 'for',
 'love',
 '-',
 'your',
 'good',
 'with',
 ',',
 'great',
 'are',
 'thanks',
 'Thanks',
 'and',
 'new',
 'some',
 'Good',
 'can',
 'You',
 'of',
 'happy',
 'nice',
 '?',
 'thank',
 'haha',
 "you're",
 'u',
 'fun',
 'will',
 'Thank',
 'see',
 'awesome',
 'on',
 'wait',
 'lol',
 '"',
 'watching',
 'Happy',
 'if',
 'best',
 'amazing',
 "I'll",
 'then',
 'we',
 'follow',
 '&amp;',
 'should',
 'glad',
 'LOL',
 'cool',
 'day',
 'morning',
 'Hey',
 'excited',
 'from',
 'everyone',
 'Hope',
 'Love',
 'song',
 'hey',
 'twitter',
 'welcome',
 'as',
 'first',
 'time',
 'beautiful',
 'what',
 'ur',
 'check',
 'Have',
 "that's",
 'sure',
 'The',
 'cute',
 'that',
 '&lt;3',
 'movie',
 'birthday',
 'sweet',
 'ya',
 'Just',
 'hope',
 ':',
 'or',
 'finally',
 'listening',
 'enjoy',
 'ready',
 'night',
 'better',
 'pretty',
 ')',
 'funny',
 'yes',
 'made',
 'lovely',
 'you!',
 'guys',
 'yay',
 'forward',
 'us',
 'sounds',
 'Morning',
 'it!',
 'just',
 'Get',
 'tweet',

In [7]:
# SVM Model feature function
def feature3(datum):
    feat = [0]*len(wordsP)
    review = datum[1]
    tokens = tweet.tokenize(review)
    for w in tokens:
        if w in wordsP:
            feat[wordIdP[w]] += 1
    feat.append(1) #offset
    return feat

In [8]:
# Baseline Model Feature Function
punctuation = set(string.punctuation)
def baseFeature(datum):
    positive_count = 0
    negative_count = 0
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    for w in r.split():
        if w in pos_words:
            positive_count += 1
        if w in neg_words:
            negative_count += 1
    print("Positive word counts: ", positive_count, "Negative word counts: ", negative_count)
    if(negative_count > positive_count):
        return 0
    else:
        return 1

In [9]:
# load json and create RNNmodel
json_file = open('RnnModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_RnnModel = model_from_json(loaded_model_json)
# load weights into new model
loaded_RnnModel.load_weights("RnnModel.h5")
print("Loaded model from disk")

Loaded model from disk


In [10]:
# load json and create CNNmodel
json_file = open('CnnModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_CnnModel = model_from_json(loaded_model_json)
# load weights into new model
loaded_CnnModel.load_weights("CnnModel.h5")
print("Loaded model from disk")

Loaded model from disk


In [11]:
#load SVM model 
clf = joblib.load('svm_model.joblib')

In [12]:
# Predict using baseline model
def predict_baseline(str_1):
    base_feature = baseFeature(str_1)
    if base_feature == 1:
        print("It's a positive sentence")
    else:
        print("It's a negative sentence")

In [13]:
# Predict using SVM model
def predict_svm(str_1):
    datum = [0,str_1]
    feat = []
    feat.append(feature3(datum))
    if clf.predict(feat)[0] == 0:
        print("It's a negative sentence")
    else:
        print("It's a positive sentence")
   

In [14]:
# Predict using LSTM_RNN model
def predict_LSTM_RNN(str_1):
    r = ' '.join([x  for x in tokenizer.tokenize(str_1) if x[0] != '@'])
    max_string = ''.join('and ' for i in range(246))
    test_x = feature_tokenizer.texts_to_sequences([r, max_string])
    test_x = pad_sequences(test_x)
    result = loaded_RnnModel.predict(np.array([test_x[0]]))
    print("Positive prob: ", result[0][0], "Negative prob: ", result[0][1])
    if(result[0][0] > result[0][1]):
        print("It's a positive sentence")
    else:
        print("It's a negtive sentence")

In [15]:
# Predict using LSTM_RNN model
def predict_CNN(str_1):
    r = ' '.join([x  for x in tokenizer.tokenize(str_1) if x[0] != '@'])
    max_string = ''.join('and ' for i in range(252))
    test_x = feature_tokenizer_cnn.texts_to_sequences([r, max_string])
    test_x = pad_sequences(test_x)
    result = loaded_CnnModel.predict(np.array([test_x[0]]))
    positive_prob = result[0][2] + result[0][3] + result[0][4]
    negtive_prob = result[0][0] + result[0][1]
    print(result[0])
    print("Positive prob: ", positive_prob, "Negative prob: ", negtive_prob)
    if(positive_prob > negtive_prob):
        print("It's a positive sentence")
    else:
        print("It's a negtive sentence")

In [16]:
# compare all four models together
def model_compare(str_l):
    print("Start to predict the sentiment of \n \"", str_l, "\"\n")
    print("Baseline model's prediction: ")
    predict_baseline(str_l)
    print("\nSVM model's prediction: ")
    predict_svm(str_l)
    print("\nLSTM RNN model's prediction: ")
    predict_LSTM_RNN(str_l)
    print("\nCNN model's prediction:" )
    predict_CNN(str_l)

In [23]:
# Test for sentences 
smaller_data[0][1]
test_string = smaller_data[15][1]
test_string = "UCSD is the best"
print(test_string)
model_compare(test_string)

UCSD is the best
Start to predict the sentiment of 
 " UCSD is the best "

Baseline model's prediction: 
Positive word counts:  2 Negative word counts:  1
It's a positive sentence

SVM model's prediction: 
It's a positive sentence

LSTM RNN model's prediction: 
Positive prob:  0.9642895 Negative prob:  0.03571048
It's a positive sentence

CNN model's prediction:
[1.6690162e-01 2.7074714e-07 2.6805029e-07 2.4749545e-07 8.3309758e-01]
Positive prob:  0.8330981 Negative prob:  0.16690189
It's a positive sentence
